## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [42]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [43]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [44]:
df_other_methods['method'].unique()

array(['ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF', 'InceptionTime',
       'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet', 'ProximityForest', 'WEASEL',
       'Catch22', 'cBOSS'], dtype=object)

In [45]:
# Set directory where the csv files are located
directory = '/home/nirojasvar/random_sast/ResultsRsast'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)

In [46]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['SmoothSubspace', 'SonyAIBORobotSurface2', 'GunPoint',
       'ItalyPowerDemand', 'ShapeletSim', 'SyntheticControl',
       'SonyAIBORobotSurface1', 'DistalPhalanxOutlineAgeGroup',
       'Chinatown', 'Coffee', 'BirdChicken'], dtype=object)

In [47]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [48]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="STC"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,BirdChicken,0.776667,0.830000,0.713333,0.755333,0.708667,0.837333,BirdChicken,0.950000
1,Chinatown,0.950865,0.962177,0.938503,0.960233,0.947833,0.963460,Chinatown,0.967930
2,Coffee,0.937143,0.997143,0.951429,0.937143,0.994286,0.992857,Coffee,1.000000
3,DistalPhalanxOutlineAgeGroup,0.613813,0.638129,0.583741,0.546619,0.546331,0.638273,DistalPhalanxOutlineAgeGroup,0.769784
4,GunPoint,0.778933,0.971111,0.784889,0.947556,0.918311,0.964978,GunPoint,0.993333
5,ItalyPowerDemand,0.890897,0.903026,0.879585,0.874921,0.885079,0.897933,ItalyPowerDemand,0.955296
6,ShapeletSim,0.944296,0.891037,0.895704,0.734593,0.631259,0.881037,ShapeletSim,1.000000
7,SmoothSubspace,0.816000,0.863022,0.821333,0.849511,0.827556,0.861689,SmoothSubspace,0.933333
8,SonyAIBORobotSurface1,0.742696,0.789063,0.743738,0.722374,0.739567,0.789040,SonyAIBORobotSurface1,0.752080
9,SonyAIBORobotSurface2,0.726828,0.889948,0.738496,0.852354,0.814453,0.890969,SonyAIBORobotSurface2,0.943337


In [49]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.706799,0.783212,0.775085,0.791331,0.796114
ACF&PACF,0.788727,0.820828,0.822242,0.829631,0.830009
Max ACF,0.683096,0.761558,0.751754,0.784855,0.776591
Max PACF,0.706962,0.779240,0.766742,0.787187,0.792193
None,0.698298,0.757902,0.750193,0.762722,0.761605
PACF,0.788717,0.811702,0.829890,0.829051,0.834072


In [50]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.697352,0.766359,0.788494,0.800485,0.799851
ACF&PACF,0.781329,0.809658,0.831284,0.833622,0.835545
Max ACF,0.652142,0.762645,0.770915,0.791305,0.780847
Max PACF,0.690333,0.783437,0.775062,0.791396,0.792094
None,0.664905,0.757710,0.757390,0.772519,0.778197
PACF,0.783304,0.811767,0.835120,0.833946,0.829294


#### Time: Subsequence Lenght Method 

In [51]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,BirdChicken,Chinatown,Coffee,DistalPhalanxOutlineAgeGroup,GunPoint,ItalyPowerDemand,ShapeletSim,SmoothSubspace,SonyAIBORobotSurface1,SonyAIBORobotSurface2,SyntheticControl
len_method,,,,,,,,,,,
ACF,81.630738,0.143859,29.515313,132.693740,39.144550,0.826132,7.645123,2.081739,0.419210,0.872094,128.598058
ACF&PACF,297.280662,0.444094,102.533573,418.627787,88.200498,2.032732,222.211859,3.407822,2.247746,3.368165,402.314136
Max ACF,2.745482,0.145839,2.160797,49.675651,4.153232,0.839920,2.640422,2.087500,0.402863,0.671550,45.391469
Max PACF,107.885490,0.162721,6.330146,29.701529,3.936128,0.915300,124.217464,2.404325,0.405555,0.545936,29.918500
None,1.853694,0.133453,1.236922,28.372396,2.529203,0.829915,1.836391,2.064745,0.300002,0.440205,28.232846
PACF,208.928276,0.387296,74.365190,293.323354,48.907430,2.043757,215.296044,3.382209,1.952652,2.593810,260.544206


In [52]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,2.173029,21.768616,40.021788,53.549746,62.318038
ACF&PACF,9.825218,94.095549,154.135075,189.627121,219.702417
Max ACF,0.467565,3.851081,9.733827,14.507606,17.897742
Max PACF,3.061148,30.067636,38.387133,36.699914,37.935412
None,0.332803,2.491777,6.154044,8.908018,10.887203
PACF,7.277284,72.805628,109.676881,136.442514,155.984182


In [53]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,1.075547,9.665356,28.430936,47.165871,93.493508
ACF&PACF,25.430795,50.948254,110.095576,174.693059,306.217697
Max ACF,0.353631,2.544149,7.358331,12.244197,23.957514
Max PACF,21.986406,22.760273,29.268765,32.491963,39.643838
None,0.237142,1.590710,4.564666,7.322503,15.058823
PACF,24.201993,40.663308,83.931859,121.264588,212.124742


#### Variance: Subsequence Lenght Method 

In [54]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
len_method,,,,,
ACF,0.043969,0.045437,0.038579,0.034686,0.036687
ACF&PACF,0.041945,0.043757,0.037865,0.039311,0.036746
Max ACF,0.053023,0.045166,0.047587,0.036747,0.043433
Max PACF,0.058240,0.046667,0.052908,0.047549,0.045974
None,0.058577,0.049471,0.054307,0.046532,0.045523
PACF,0.037621,0.043237,0.034179,0.037765,0.040664


In [55]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,1,10,30,50,100
len_method,,,,,
ACF,0.050620,0.040047,0.041946,0.035009,0.033872
ACF&PACF,0.047935,0.038692,0.039680,0.037886,0.036417
Max ACF,0.052810,0.046975,0.055402,0.038075,0.039033
Max PACF,0.062347,0.048135,0.057853,0.043975,0.041687
None,0.063803,0.049842,0.054123,0.046969,0.045337
PACF,0.043352,0.045162,0.036309,0.037373,0.031771


#### Another Results

In [56]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,4500,4500,4500
unique,11,6,150
top,SmoothSubspace,ACF&PACF,ACF&PACF: n_random_points=1 nb_inst_per_class=1
freq,450,750,30


In [57]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
BirdChicken,14.590090
Chinatown,0.029526
Coffee,1.500986
DistalPhalanxOutlineAgeGroup,13.227701
GunPoint,3.893147
ItalyPowerDemand,0.155995
ShapeletSim,11.955152
SmoothSubspace,0.321424
SonyAIBORobotSurface1,0.119334


In [58]:
df_result['time'].sum()/(60*60)

64.61608327680165

#### Generate CD Diagram Tunning Hyperparameter

In [59]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [60]:
from cd_diagram.cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='cd-diagram_overall_tunning.png')



/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not 

In [61]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
Max ACF: n_random_points=1 nb_inst_per_class=1,131.909091
Max ACF: n_random_points=30 nb_inst_per_class=1,130.727273
Max PACF: n_random_points=30 nb_inst_per_class=1,126.090909
Max ACF: n_random_points=10 nb_inst_per_class=1,124.409091
ACF: n_random_points=1 nb_inst_per_class=1,121.090909
...,...
ACF&PACF: n_random_points=100 nb_inst_per_class=50,38.454545
ACF&PACF: n_random_points=100 nb_inst_per_class=30,38.045455
ACF&PACF: n_random_points=50 nb_inst_per_class=50,37.681818
PACF: n_random_points=100 nb_inst_per_class=30,36.318182


In [63]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [72]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=10 nb_inst_per_class=100'], dtype='object')

In [64]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.717509,0.738256,0.830947,0.826298,0.830625
10,0.765374,0.831898,0.836783,0.831645,0.838439
30,0.782474,0.822437,0.834750,0.834596,0.836951
50,0.824607,0.828411,0.817120,0.839671,0.838345
100,0.816680,0.827286,0.836819,0.835897,0.833362


In [65]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.070920,0.069734,0.025881,0.037051,0.029700
10,0.052213,0.029483,0.035178,0.040476,0.037432
30,0.039635,0.046783,0.035078,0.042022,0.038178
50,0.020471,0.034556,0.058429,0.040917,0.039912
100,0.024563,0.037625,0.039700,0.041406,0.043528


##### Overall time complexity

In [66]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000160,0.001823,0.000126,0.005264,0.007402
Coffee,0.001159,0.119599,0.000517,1.587549,1.708893
DistalPhalanxOutlineAgeGroup,0.000734,0.040038,0.010659,6.925591,6.977130
GunPoint,0.000717,0.042117,0.001210,1.425890,1.470008
ItalyPowerDemand,0.000144,0.003278,0.000300,0.030122,0.033879
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000118,0.003916,0.000706,0.052012,0.056797
SonyAIBORobotSurface1,0.000386,0.004721,0.000210,0.032119,0.037462


In [67]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.060866,0.471957,0.549030,0.533232,0.504148
10,0.080412,0.733682,1.026237,1.157533,1.247825
30,0.134850,1.303655,2.070501,2.628801,3.036823
50,0.204147,1.954881,3.636967,3.883660,4.878099
100,0.338494,3.377119,5.561855,7.599034,8.641640


##### Calculate weights time complexity

In [68]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.038623,0.035392,0.038248,0.038599,0.040482
10,0.040802,0.036379,0.042809,0.043718,0.047813
30,0.045281,0.042171,0.047317,0.050861,0.049500
50,0.045130,0.046597,0.046168,0.048084,0.050724
100,0.049255,0.043274,0.048704,0.050723,0.047308


##### Finding subsequences time complexity

In [69]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,2.771953,26.497612,29.715038,27.739550,25.221798
10,2.972292,26.651872,29.993084,28.287877,25.468365
30,2.708080,25.853590,30.846149,33.629055,32.052449
50,3.226751,32.169176,34.043890,29.988277,29.465230
100,3.219135,30.619440,27.939760,42.735333,30.321856


##### Transform Dataset time complexity

In [70]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.821793,1.741718,3.149174,4.170493,4.944670
10,1.775288,17.271305,31.465248,41.048449,49.261780
30,5.286499,52.252682,93.215017,123.775977,148.392660
50,8.912045,84.962311,155.215787,202.715396,243.262043
100,16.966931,171.826543,304.796668,411.854548,486.492714


##### Classifier time complexity

In [71]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.016988,0.041675,0.038327,0.043901,0.040754
10,0.035332,0.060109,0.071077,0.069446,0.088762
30,0.049704,0.068720,0.116855,0.266788,1.707749
50,0.061925,0.110320,28.906099,0.259070,19.899517
100,0.070121,0.131620,0.915979,1.288700,1.622938
